Reading and Exploring Data

In [10]:
# Loading Data
with open ('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [11]:
# Getting Length of Dataset
print("Length of dataset in characters: ", len(text))

Length of dataset in characters:  1115394


In [12]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [13]:
# Getting all the unique characters (in a set) that occur in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Tokenizing and Splitting Data

In [14]:
# Tokenization
# Creating a mapping from char to int
stoi = { ch:i for i, ch in enumerate(chars) } # Lookup Table from char to int
itos = { i:ch for i, ch in enumerate(chars) } # Lookup table from int to char

# Encoder:- Take str output a list of int
encode = lambda s: [stoi[c] for c in s]
# Decoder:- Take a list of int output a str
decode = lambda l: ''.join(itos[i] for i in l)

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [15]:
# Tokenizing the dataset
import torch

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [16]:
# Data Splitting (90-10: Train-val Split)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

Data Loader

In [17]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [18]:
x = train_data[:block_size]
y= train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]

    print(f"When input is {context} the target is {target}")

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [19]:
torch.manual_seed(1337)
batch_size = 4 # Independent Sequences
block_size = 8 # Maximum Context Length

def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('###########')

for b in range(batch_size): # batch dimensions
    for t in range(block_size): # time dimensions
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the output is {target}")  # Context is taken from xb tensor and target from yb tensor
    

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
###########
When input is [24] the output is 43
When input is [24, 43] the output is 58
When input is [24, 43, 58] the output is 5
When input is [24, 43, 58, 5] the output is 57
When input is [24, 43, 58, 5, 57] the output is 1
When input is [24, 43, 58, 5, 57, 1] the output is 46
When input is [24, 43, 58, 5, 57, 1, 46] the output is 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the output is 39
When input is [44] the output is 53
When input is [44, 53] the output is 56
When input is [44, 53, 56] the output is 1
When input is [44, 53, 56, 1] the output is 58
When input is [44, 53, 56, 1, 58] the o